In [17]:
import os
#from dotenv import load_dotenv
#load_dotenv()
from sqlalchemy.sql import func
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from sqlalchemy import Column, Integer, String, Float
import numpy as np
import pandas as pd
import os


#################################################
# Database Setup
#################################################
engines = create_engine("sqlite:///Datasets//Weather_3.db")

# Save reference to the table

conn = engines.connect()
Base = automap_base()
Base.prepare(engines, reflect=True)
session = Session(conn)
Base.classes.keys()

['State_Code', 'Weather_Raw', 'weather_data_project']

In [18]:
# Performs database schema inspection
#insp = sqlalchemy.inspect(engines)
#print(insp.get_table_names())

In [19]:
Wtr = Base.classes.weather_data_project
wtr_data = session.query(Wtr).statement
#St_Cd = Base.classes.State_Code
#join = session.query( Wtr , St_Cd ).filter(Wtr.State == St_Cd.Code).statement

In [20]:
df = pd.read_sql_query(wtr_data, session.bind)
df

,State_Code,State,Year,Jan_Avg,Feb_Avg,Mar_Avg,Apr_Avg,May_Avg,Jun_Avg,Jul_Avg,Aug_Avg,Sep_Avg,Oct_Avg,Nov_Avg,Dec_Avg
0,1,Alabama,1895,43,37,54,63,69,77,79,79,78,59,53,45
1,1,Alabama,1896,43,47,52,68,76,77,81,82,76,63,57,46
2,1,Alabama,1897,41,51,60,62,69,81,81,79,75,67,54,47
3,1,Alabama,1898,49,46,59,58,74,80,80,79,75,61,50,43
4,1,Alabama,1899,43,40,55,61,76,80,80,81,73,66,55,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6237,50,Alaska,2018,16,18,24,32,41,49,54,51,46,38,27,18
6238,50,Alaska,2019,17,20,30,33,44,52,56,53,46,35,28,18
6239,50,Alaska,2020,5,14,19,31,44,49,53,52,44,33,21,20
6240,50,Alaska,2021,20,12,18,29,41,50,52,50,42,32,14,14
